In [ ]:
# import packages
import logging
import os

import dotenv

import schwabdev

In [ ]:
# set logging level
logging.basicConfig(level=logging.INFO)

# load environment variables and make client
dotenv.load_dotenv()
client = schwabdev.Client(os.getenv('app_key'), os.getenv('app_secret'), os.getenv('callback_url'))

# Basic API calls

In [ ]:
# get account number and hashes for linked accounts
linked_accounts = client.account_linked().json()
print(linked_accounts)
# select the first account to use for orders
account_hash = linked_accounts[0].get('hashValue')

In [ ]:
# get positions for selected account
print(client.account_details(account_hash, fields='positions').json())

In [ ]:
# get a list of quotes
print(client.quotes(['AAPL', 'AMD']).json())

In [ ]:
# get an option chain
print(client.option_expiration_chain('AAPL').json())

# Order example

In [ ]:
# place an order for INTC at limit price $10.00
order = {
    'orderType': 'LIMIT',
    'session': 'NORMAL',
    'duration': 'DAY',
    'orderStrategyType': 'SINGLE',
    'price': '10.00',
    'orderLegCollection': [
        {
            'instruction': 'BUY',
            'quantity': 1,
            'instrument': {'symbol': 'INTC', 'assetType': 'EQUITY'},
        }
    ],
}
resp = client.order_place(account_hash, order)
print(f'Response code: {resp}')

# get the order ID - if order is immediately filled then the id might not be returned
order_id = resp.headers.get('location', '/').split('/')[-1]
print(f'Order id: {order_id}')

In [ ]:
# cancel the order
print(client.order_cancel(account_hash, order_id))

# Streaming example

In [ ]:
# create streamer
streamer = client.stream

In [ ]:
# create a list to store responses
responses = []


def add_to_list(message):
    responses.append(message)

In [ ]:
# start stream and send request
streamer.start(add_to_list)
streamer.send(streamer.level_one_equities('AMD', '0,1,2,3,4,5,6,7,8'))

In [ ]:
# check responses
print(responses)

In [ ]:
# stop stream
streamer.stop()